<a href="https://colab.research.google.com/github/furtman/WQTC25_workshop_IMTR/blob/main/Ex_geospatial/water_quality_visualization_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PCW01: From Data Management to Data Analysis — A Technical Deep-Dive**
AWWA WQTC 2025, November 9-13th, Tacoma WA
## **Water Quality Data Visualization using Google Colab and Python**

---

This notebook walks through building and running an interactive data visualization dashboard inside a **Google Colab** workspace using the **Python** programming language. Users will learn how to load, inspect, and analyze data from a spreadsheet, generate static and interactive visualizations of time series plots and maps, and compile the visualizations into a PDF report and an easy-to-use dashboard that synthesizes data into single webpage.

The data used in this notebook is publicly available at Thurston County:
1.   Water Quality Data: https://github.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/blob/main/wqp_data.csv
2.   Stream Site Locations and Aquatic Use Data: https://github.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/blob/main/outputs/streams_sites.csv

An example of a fully-functional WQ data dashboard created using Shiny and R - Thurston County Streams Water Quality Dashboard: https://www.thurstoncountywa.gov/streams-water-quality-dashboard

---

Notebook Table of Contents:
1.   Setup Python Runtime
2.   Load Python Libraries
3.   Upload Files to Google Colab
4.   Load and Inspect Data
5.   Data Cleaning and Processing
6.   Generating Summary Statistics
7.   Generating Plots
8.   Generating Multiple Plots
9.   Generating Maps



---


## 1. Setup Python Runtime
Prior to installing any packages, change the ***runtime type*** to ***Python 3*** if it isn't already selected. Google Colab comes with several popular python packages pre-installed, such as *pandas* and *matplotlib*, which are commonly used to .





---


## 2. Load Python Libraries
In order to use the functions within each library, we need to load them into the notebook.

In [ ]:
# Load in the Python libraries that will be used in this notebook
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import plotly.express as px
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import geopandas as gpd
#import contextily as ctx
#from jupyter_dash import JupyterDash
#from dash import dcc, html, Input, Output



---


## 3. Upload Files to Google Colab
Since Google Colab is run on Linux cloud machines, we need to transfer the files from our PC (Windows, macOS, etc.) to the Google Colab workspace.

In [ ]:
# Choose files to upload; By default, uploaded files are saved in the "Files/content/" folder within the Google Colab workspace
# from google.colab import files
uploaded_files = files.upload()

In [ ]:
# Or we can download directly from the internet
# URLs of water quality data (Thurston County)
sites_url = "https://raw.githubusercontent.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/main/outputs/streams_sites.csv"
data_url = "https://raw.githubusercontent.com/HerreraEnvironmental/23-08082-000-TC-WQ-Dashboard/main/wqp_data.csv"



---


## 4. Load and Inspect Data

Read in the CSV files to the notebook as variables called `sites` and `wq_data`.

In [ ]:
# Read in stream site location data and inspect
# import pandas as pd
sites = pd.read_csv("streams_sites.csv")
print('This is the stream site location data:') # output a message to the console
sites # show data

In [ ]:
# Read in water quality data and inspect
wq_data = pd.read_csv("wqp_data.csv")
print("This is the water quality data associated with each stream site:")
wq_data

In [ ]:
# Show some general info about the data types of each column in the water quality data
wq_data.info()

In [ ]:
# Show the unique stream site locations and parameter names
print("The unique stream site locations are:", sites['SITE_NAME'].unique())
print("\n") # add a line break so it's easier to see the different outputs
print("The unique parameter names are:", wq_data['parameter'].unique())



---


## 5. Data Cleaning and Processing

In [ ]:
# In order to conduct timeseries analysis and plot the WQ data, we need to change the 'date_time' column to a datetime data type and sort by date
wq_data['date_time_new'] = pd.to_datetime(wq_data['date_time'], format='mixed')
wq_data = wq_data.sort_values(by='date_time_new') # sort values by datetime
wq_data.info()

In [ ]:
# In order to conduct geospatial analysis and generate maps, we need to join/merge the WQ data with the stream site location data
merged_data = pd.merge(
    wq_data,
    sites[['SITE_NAME', 'LAT', 'LON', 'AquaticLifeUse']], # keep only the site name, latitude, longitude, and aquatic life use columns
    on='SITE_NAME', # merge the dataframes based on a common column
    how='left' # keep all rows in the wq_data and only matching rows in the sites data
)

merged_data.columns # show the dataframe column names



---

## 6. Generating Summary Statistics

In [ ]:
# Generate some basic summary stats of observed WQ values for each site and parameter pairing
summary_stats = merged_data.groupby(['SITE_NAME', 'parameter'])['value'].agg(['count','mean','median','min','max']).reset_index()
summary_stats

In [ ]:
# View summary stats for a specific site
summary_stats[summary_stats['SITE_NAME'] == 'Deschutes @ Tumwater Falls Park']



---


## 7. Generating Plots

### Static Plots

In [ ]:
# Generate a timeseries plot of a WQ parameter for a specific site
# Filter the data for a specific site name and parameter (e.g., water temperature)
site_name = 'Deschutes @ Tumwater Falls Park'
param_name = 'Temperature, water'
site_temp_data = merged_data[ # filter out data for specific parameter and site
    (merged_data['SITE_NAME'] == site_name) &
    (merged_data['parameter'] == param_name)
    ].copy() # save to a new dataframe
site_temp_data = site_temp_data.sort_values(by='date_time_new') # sort the data by date/time
site_temp_data

In [ ]:
# Basic line plot
# import matplotlib.pyplot as plt
plt.plot(site_temp_data['date_time_new'], site_temp_data['value']) # x, y
plt.xlabel('Date') # x-axis label
plt.ylabel(param_name) # y-axis label
plt.title(f'Parameter: {param_name} | Site: {site_name}') # plot title
plt.show()

In [ ]:
# Fancy plot with summary stats and trend line
# from scipy.stats import linregress

# Convert temperature from C to F
site_temp_data['value_F'] = site_temp_data['value'] * 9/5 + 32

# Calculate stats
mean_temp = site_temp_data['value_F'].mean()
min_temp = site_temp_data['value_F'].min()
max_temp = site_temp_data['value_F'].max()

# Linear regression for trend
# x as ordinal dates
x = site_temp_data['date_time_new'].map(lambda dt: dt.toordinal())
y = site_temp_data['value_F']
slope, intercept, r_value, p_value, std_err = linregress(x, y)

# Calculate trend line y values
slope_per_year = slope * 365.25 # convert to per year
trend_line = slope * x + intercept

# Compute R² and p-value significance
r_squared = r_value ** 2
if p_value < 0.05:
  significance = "Significant (p < 0.05)"
else:
  significance = "Not Significant (p > 0.05)"
#significance = "Significant" if p_value < 0.05 else "Not Significant" # how the cool kids do it

# List of stats that will be added to the plot
stats_text = (
    f"Max: {max_temp:.1f}°F\n"
    f"Min: {min_temp:.1f}°F\n"
    f"Mean: {mean_temp:.1f}°F\n"
    f"Trend slope: {slope_per_year:.3f}°F/year\n"
    f"R²: {r_squared:.3f}\n"
    f"p-value: {p_value:.4f}\n"
    f"{significance}"
)

# Plotting
plt.figure(figsize=(14, 6)) # width, height
plt.plot(site_temp_data['date_time_new'], site_temp_data['value_F'], # x, y
         marker='o', markersize=3, # marker style
         linestyle='-', linewidth=1, # line style
         label='Observed Data') # legend label
plt.plot(site_temp_data['date_time_new'], trend_line, # trend line
         linestyle='--', linewidth=2, color='red', # line style
         label='Trend Line') # legend label

# Add stats to plot
plt.text(0.01, 0.98, stats_text, transform=plt.gca().transAxes,
         verticalalignment='top', fontsize=10, bbox=dict(facecolor='white', alpha=0.5, edgecolor='gray'))

# Plot beautification
plt.grid(True, linestyle='--', alpha=0.8) # background grid
plt.title(f'{param_name} at {site_name}', fontsize=14, fontweight='bold') # plot title
plt.ylabel("Water Temperature (°F)", fontsize=12) # y-axis label
plt.xticks(rotation=45) # rotate the x-axis labels
plt.legend() # add legend
plt.tight_layout() # auto adjust spacing to prevent overlapping
plt.show()

### Interactive Plots

In [ ]:
# Generate a basic interactive plot of water temperature at every site on the Deschutes River
# import plotly.express as px

# Filter water temperature data and site names that begin with "Deschutes"
filtered_data = merged_data[
    (merged_data['parameter'] == param_name) & # 'Temperature, water'
    (merged_data['SITE_NAME'].str.startswith('Deschutes'))
    ].copy()

# Sort by datetime
filtered_data = filtered_data.sort_values(by='date_time_new')

# Plotting
fig = px.line(
    filtered_data,
    x='date_time_new', # x data
    y='value', # y data
    color='SITE_NAME', # symbolize site name by color
    title='Deschutes River Water Temperature'
)

# Layout styling for comparison
fig.update_layout(hovermode='x unified') # enables value hovering for easy comparison

fig.show()

In [ ]:
# Generate a fancy interactive plot of all WQ parameters at every site on the Deschutes River

# Filter site names that begin with "Deschutes" and sort by datetime
deschutes_data = merged_data[
    (merged_data['SITE_NAME'].str.startswith('Deschutes'))
    ].sort_values(by='date_time_new').copy()

# Plotting
fig_deschutes = px.line(
    deschutes_data,
    x='date_time_new',
    y='value',
    color='SITE_NAME',
    facet_col='parameter', # each parameter gets its own plot
    facet_col_wrap=3, # 3 plots per row
    title='Deschutes River Water Quality Parameters',
    height=800, # figure height
    labels={ # customize label names for data hover
        'date_time_new': 'Date',
        'value': 'Measured Value',
        'SITE_NAME': 'Site Name',
        'unit': 'Unit'
    },
    hover_data={ # customize label values for data hover
        'date_time_new': '|%m-%d-%Y', # reformat the datetime to mm/dd/yyyy
        'value': ':.2f', # value with 2 decimals
        'SITE_NAME': True, # inlcude site name in hover
        'unit': True,
        'parameter': False # do not include parameter name in hover
    }
)

# Improve layout styling
fig_deschutes.update_layout(
    hovermode='x unified',
    showlegend=True,
    legend_title_text='Site Name', # update the legend title
    title_font_size=18, # adjust plot title font size
    margin=dict(l=40, r=40, t=80, b=40) # adjust plot margins (left, right, top, bottom)
)

# Make each facet plot's y-axis values independent from each other (needed since parameters have different measurement units)
fig_deschutes.update_yaxes(title='', matches=None) # don't show y-axis label
fig_deschutes.update_xaxes(title='') # don't show x-axis label

# Clean up facet titles: remove "parameter_unit=" prefix
for annotation in fig_deschutes.layout.annotations:
    annotation.text = annotation.text.replace("parameter=", "")

fig_deschutes.show()

## 8. Generating Multiple Plots
Up until this point, Excel would have been able to handle everything we just did (i.e., look at the data, filter by parameter and site, create plots). With Python (and programming in general), we can take this a step further by automating those redundant processes that are done in Excel. For example, we can write a script to plot a timeseries of all WQ parameters at each stream site location and then save to a PDF.

In [ ]:
# Generate a fancy, multi-facet plot of all WQ parameters for each stream site location and save to a single PDF (takes approx. 4-5 minutes to run)
# import numpy as np
# from matplotlib.backends.backend_pdf import PdfPages

# Output file
output_pdf = "WaterQuality_SiteParameter_Trends.pdf"

# Create PDF file to save all plots; each site gets a single PDF page
with PdfPages(output_pdf) as pdf:
    # Loop through each site
    for site_name, site_df in merged_data.groupby('SITE_NAME'):
        parameters = site_df['parameter'].unique()

        # Setup multi-facet plot layout
        n_params = len(parameters)
        ncols = 2
        nrows = int(np.ceil(n_params / ncols))
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12, nrows * 4))
        axes = axes.flatten()

        # Loop through each parameter, apply regression, calculate stats, and plot
        for i, param_name in enumerate(parameters):
            ax = axes[i]
            param_df = site_df[site_df['parameter'] == param_name].copy()
            param_df = param_df.sort_values(by="date_time_new")

            # Convert temperature from C to F
            if "temp" in param_name.lower():
                param_df['value_plot'] = param_df['value'] * 9/5 + 32
                y_label = "Temperature (°F)"
            else:
                param_df['value_plot'] = param_df['value']
                y_label = f"{param_name} ({param_df['unit'].iloc[0] if 'unit' in param_df.columns else ''})"

            # Data for regression
            x = param_df['date_time_new'].map(lambda dt: dt.toordinal())
            y = param_df['value_plot']
            n_points = len(y)

            # Check data sufficiency
            if n_points >= 3:
                slope, intercept, r_value, p_value, std_err = linregress(x, y)
                slope_per_year = slope * 365.25
                r_squared = r_value ** 2
                trend_line = slope * x + intercept
                significance = "Significant (p < 0.05)" if p_value < 0.05 else "Not Significant"
            else:
                slope_per_year = np.nan
                r_squared = np.nan
                p_value = np.nan
                trend_line = [np.nan] * len(x)
                significance = "Insufficient data for regression"

            # Summary statistics
            mean_val = y.mean()
            min_val = y.min()
            max_val = y.max()

            # Stats text
            if np.isnan(p_value):
                p_str = "N/A"
            else:
                p_str = f"{p_value:.4f}"

            stats_text = (
                f"n = {n_points}\n"
                f"Max: {max_val:.1f}\n"
                f"Min: {min_val:.1f}\n"
                f"Mean: {mean_val:.1f}\n"
                f"Trend: {slope_per_year:.3f}/yr\n"
                f"R²: {r_squared:.3f}\n"
                f"p: {p_str}\n"
                f"{significance}"
            )

            # Plotting
            ax.plot(param_df['date_time_new'], y, marker='o', markersize=3,
                    linestyle='-', linewidth=1, label='Observed')
            ax.plot(param_df['date_time_new'], trend_line, linestyle='--',
                    linewidth=2, color='red', label='Trend')
            ax.text(0.01, 0.98, stats_text, transform=ax.transAxes, fontsize=9,
                    verticalalignment='top',
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))
            ax.set_title(f"{param_name}", fontsize=11, fontweight='bold')
            ax.set_ylabel(y_label)
            ax.grid(True, linestyle='--', alpha=0.6)
            ax.legend(fontsize=8)
            ax.tick_params(axis='x', rotation=45)

        # Remove unused subplots
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        # Layout and save
        fig.suptitle(f"Water Quality Trends at {site_name}", fontsize=14, fontweight='bold')
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        pdf.savefig(fig)
        plt.close(fig)

print(f"All plots saved to: {output_pdf}")

## 9. Generating Maps